In [11]:
from google.colab import drive
drive.mount('/drive')

import numpy as np
import pandas as pd

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [2]:
!unzip "/drive/MyDrive/Colab Notebooks/nature.zip"  -d "/drive/MyDrive/Colab Notebooks/nature"

Archive:  /drive/MyDrive/Colab Notebooks/nature.zip
   creating: /drive/MyDrive/Colab Notebooks/nature/nature/
   creating: /drive/MyDrive/Colab Notebooks/nature/nature/seg_test/
   creating: /drive/MyDrive/Colab Notebooks/nature/nature/seg_test/seg_test/
   creating: /drive/MyDrive/Colab Notebooks/nature/nature/seg_test/seg_test/buildings/
  inflating: /drive/MyDrive/Colab Notebooks/nature/nature/seg_test/seg_test/buildings/20057.jpg  
  inflating: /drive/MyDrive/Colab Notebooks/nature/nature/seg_test/seg_test/buildings/20060.jpg  
  inflating: /drive/MyDrive/Colab Notebooks/nature/nature/seg_test/seg_test/buildings/20061.jpg  
  inflating: /drive/MyDrive/Colab Notebooks/nature/nature/seg_test/seg_test/buildings/20064.jpg  
  inflating: /drive/MyDrive/Colab Notebooks/nature/nature/seg_test/seg_test/buildings/20073.jpg  
  inflating: /drive/MyDrive/Colab Notebooks/nature/nature/seg_test/seg_test/buildings/20074.jpg  
  inflating: /drive/MyDrive/Colab Notebooks/nature/nature/seg_test/se

In [25]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Rescaling
from keras.utils import to_categorical, image_dataset_from_directory
from keras import initializers
from keras import regularizers
import cv2
import random
import os

In [4]:
DIRECTORY = '/drive/MyDrive/Colab Notebooks/nature/nature/seg_train/seg_train'
CATEGORIES = ['buildings','forest','glacier','mountain','sea','street']


os. path. join combines path names into one complete path. 

This means that you can merge multiple parts of a path into one, instead of hard-coding every path name manually.

Python method listdir() returns a list containing the names of the entries in the directory given by path. The list is in arbitrary order.

has context menu

In [16]:
data=[]

for category in CATEGORIES:
  path=os.path.join(DIRECTORY, category)
  for img in os.listdir(path):
    img_path = os.path.join(path, img)
    label=CATEGORIES.index(category)
    #It specifies to load an image in grayscale mode. Can red a color image file in grayscale (black and white).
    arr = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    new_arr = cv2.resize(arr,(120,120))
    data.append([new_arr,label])

In [6]:
len(data)

2700

In [15]:
random.shuffle(data)
X =[]
Y =[]

for features,label in data:
  X.append(features)
  Y.append(label)

X = np.array(X)
Y = np.array(Y)
X = X/255
X = X.reshape(-1,120,120,1)
print(X.shape)
Y.shape


(2700, 120, 120, 1)


(2700,)

In [14]:
DIRECTORY2 = '/drive/MyDrive/Colab Notebooks/nature/nature/seg_test/seg_test'
CATEGORIES2 = ['buildings','forest','glacier','mountain','sea','street']

In [17]:
data_test=[]

for category in CATEGORIES2:
  path=os.path.join(DIRECTORY2, category)
  for img in os.listdir(path):
    img_path = os.path.join(path, img)
    label=CATEGORIES2.index(category)
    #It specifies to load an image in grayscale mode. Can red a color image file in grayscale (black and white).
    arr = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    new_arr = cv2.resize(arr,(120,120))
    data_test.append([new_arr,label])

In [18]:
random.shuffle(data_test)
X_test =[]
Y_test =[]

for features,label in data_test:
  X_test.append(features)
  Y_test.append(label)

X_test = np.array(X_test)
Y_test = np.array(Y_test)
X_test = X_test/255
X_test = X_test.reshape(-1,120,120,1)
X_test.shape
Y_test.shape

(536,)

In [27]:
from keras import regularizers
# using CNN

L1 = 40
L2 = 40
L3 = 50

model=Sequential()
model.add(Conv2D(L1, (3,3), activation ='relu', input_shape= (120,120,1), padding='same', kernel_initializer = 'random_normal',kernel_regularizer = regularizers.l1(0.01)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(L2,(3,3), activation = 'relu', padding='same'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(L3, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(6,activation='softmax'))

In [28]:
model.compile(loss='sparse_categorical_crossentropy', optimizer ='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X, Y, epochs=50, batch_size=100, validation_data=(X_test, Y_test))

Epoch 1/50
27/27 [==============================] - 61s 2s/step - loss: 1.7049 - accuracy: 0.3585 - val_loss: 1.5444 - val_accuracy: 0.4160
Epoch 2/50
16/27 [================>.............] - ETA: 19s - loss: 1.4867 - accuracy: 0.4531